<a href="https://colab.research.google.com/github/VittorioRossi/NLP-content-moderation/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
%%capture
!pip install spacy[transformers]
!python3 -m spacy download en_core_web_md

# Model creation

In [49]:
import pandas as pd 
import numpy as np
from tqdm import tqdm
import spacy

In [ ]:
from google.colab import drive
drive.mount('drive')

In [6]:
train_data = pd.read_csv("/content/drive/MyDrive/Datasets/text-moderation/merged_train.csv")
test_data = pd.read_csv("/content/drive/MyDrive/Datasets/text-moderation/merged_test.csv")

In [10]:
LABELS = ["quality", "toxic" ,"spam"]

nlp = spacy.load('en_core_web_md')

doc_bin_train = spacy.tokens.DocBin()
doc_bin_test = spacy.tokens.DocBin()


for text, lab in tqdm(nlp.pipe(zip(list(train_data['text']), list(train_data['label'])), as_tuples=True)):
    for l in LABELS: text.cats[l] = 0 
    text.cats[LABELS[lab]] = True
    doc_bin_train.add(text)

for text, lab in tqdm(nlp.pipe(zip(list(test_data['text'][:10_000]), list(test_data['label'][:10_000])), as_tuples=True)):
    for l in LABELS: text.cats[l] = 0 
    text.cats[LABELS[lab]] = True
    doc_bin_test.add(text)

doc_bin_train.to_disk('./train.spacy')
doc_bin_test.to_disk('./test.spacy')

18000it [03:04, 97.74it/s] 
10000it [01:55, 86.39it/s]


In [13]:
%%bash
python -m spacy init fill-config ./base_config.cfg ./config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


2023-02-11 09:27:48.635543: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-11 09:27:48.635681: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-11 09:27:48.635700: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2023-02-11 09:27:50.363486: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to c

In [14]:
!python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./test.spacy --output ./output

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-02-11 09:27:56.454527: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-11 09:27:56.454679: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-11 09:27:56.454705: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentione

In [47]:
import shutil
shutil.copytree('/content/output/model-best', "/content/drive/MyDrive/Datasets/text-moderation/best-model", dirs_exist_ok=True)

'/content/drive/MyDrive/Datasets/text-moderation/best-model'

In [ ]:
shutil.copytree('/content/output/model-best', "/content/drive/MyDrive/Datasets/text-moderation/best-model", dirs_exist_ok=True)

In [15]:
nlp_test = spacy.load('/content/output/model-best')

In [36]:
balanced_test = pd.concat((
    test_data[test_data.label == 0][:1_300], 
    test_data[test_data.label == 1][:1_300],
    test_data[test_data.label == 2][:1_300],
))

In [37]:
pred = []
for text in tqdm(nlp_test.pipe(list(balanced_test['text']))):
  pred.append(np.argmax([text.cats["quality"], text.cats["toxic"], text.cats["spam"]]))

3900it [00:11, 351.49it/s]


In [38]:
from sklearn.metrics import f1_score, accuracy_score, recall_score, confusion_matrix

In [39]:
accuracy_score(balanced_test.label, pred)

0.8874358974358975

In [43]:
f1_score(balanced_test.label, pred,  average = "weighted")

0.8882480475094354

In [45]:
confusion_matrix(balanced_test.label, pred)

array([[1195,   90,   15],
       [ 248, 1046,    6],
       [  61,   19, 1220]])